# Algorithm Performance Tracking

This is going to be a markdown file where I can access the performance tracking database of my algorithms and interactively work through them to look at perfomrance metrics, commission payments, win and loss streaks etc.

The first thing that we need to do is bring in our main source code for River Rose as well as the KPI source code. Both numpy and pandas are natively in the RiverRose source code however, I will be doing a lot of exploratory analysis, so I am giong to bring them in here as well. I will also bring in Altair for some basic plotting.

## Import Libraries

In [1]:
import RiverRose.graphing as rrg
import RiverRose.administrative as rra
import numpy as np
import pandas as pd
import sqlalchemy as pg

Now that we have our libraries. Let's connect to the database to pull in our files. The first code block that I want is to see which algorithms actually have data (i.e. have been actively traded). From there, we will be able to pick and choose the algorithms we want to track performance one. We will get an output list of tables but they are in the format of tuples. This means we can either unpack them in a for loop with two variables (for table,_ in tables). Or we can index them using a format like tables[0][0]. Once we have some of the information listed, let's pull the data into a dictionary of dataframes based on the name of the algorithm so that we can access the information and start our exploratory data analysis

## Dynamic Variables

In [2]:
#let's Choose which algo we are going to be working with today OR 'None'
algo = input("What algoirthm are you tracking today? ")

#Let's also put a way to filter by Algorithm Version
algo_version = 'None' #'None' or 1, 2, etc.

## Database Import and Dataframe Construction

In [ ]:
DB_NAME = input('What database will you be using today? ')
DB_USER = input('Please enter your database username: ')
DB_PASSWORD = input('Please enter your database password: ')
DB_HOST = input("Please include the url for your database: ")
DB_PORT = '5432'

#Let's use SQLalchemy to create our engine for database connection, lets remove the account and the algorithm from the query though
engine = pg.create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

sql_query = """SELECT performance.id, performance.date,  tickers.ticker, performance.margin, 
                        direction.direction, performance.gross_profit, performance.commissions, performance.algo_version 
                FROM performance
                JOIN tickers ON performance.ticker_id = tickers.id
                JOIN direction ON performance.direction_id = direction.id 
                ORDER BY performance.id;"""

sql_query2 = """SELECT DISTINCT ON (performance.date) performance.id, performance.date, 
                                tickers.ticker, performance.margin, direction.direction, performance.gross_profit, 
                                performance.commissions, performance.algo_version 
                FROM performance 
                JOIN tickers ON performance.ticker_id = tickers.id
                JOIN direction ON performance.direction_id = direction.id 
                ORDER BY performance.date, performance.id;"""

performance = pd.read_sql(sql_query,engine)
performance_2 = pd.read_sql(sql_query2,engine)

In [7]:
performance.head()

,id,date,ticker,margin,direction,gross_profit,commissions,algo_version
0,57,2025-01-06,MNQ,2027.0,LONG,-126.0,1.24,1
1,58,2025-01-07,MNQ,2027.0,SHORT,234.5,1.24,1
2,59,2025-01-08,MNQ,2027.0,LONG,-124.5,1.24,1
3,60,2025-01-10,MNQ,2027.0,SHORT,299.0,1.24,1
4,61,2025-01-13,MNQ,2027.0,SHORT,-90.5,1.24,1


In [8]:
performance_2.head()

,id,date,ticker,margin,direction,gross_profit,commissions,algo_version
0,57,2025-01-06,MNQ,2027.0,LONG,-126.0,1.24,1
1,58,2025-01-07,MNQ,2027.0,SHORT,234.5,1.24,1
2,59,2025-01-08,MNQ,2027.0,LONG,-124.5,1.24,1
3,60,2025-01-10,MNQ,2027.0,SHORT,299.0,1.24,1
4,61,2025-01-13,MNQ,2027.0,SHORT,-90.5,1.24,1


Now that we have our data in dataframes and our database is closed, we can start to look at some basic information. I am going to start by making sure the formatting is correct. We have no missing values, the column headers make sense etc.

## Statistical Data Analysis

In [9]:
#Let's calcualte some additonal metrics quickly for analysis
performance['net_profit'] = performance['gross_profit'] - performance['commissions']
performance['return'] = np.round(performance['net_profit']/performance['margin'] *100,2)

performance.head()

,id,date,ticker,margin,direction,gross_profit,commissions,algo_version,net_profit,return
0,57,2025-01-06,MNQ,2027.0,LONG,-126.0,1.24,1,-127.24,-6.28
1,58,2025-01-07,MNQ,2027.0,SHORT,234.5,1.24,1,233.26,11.51
2,59,2025-01-08,MNQ,2027.0,LONG,-124.5,1.24,1,-125.74,-6.20
3,60,2025-01-10,MNQ,2027.0,SHORT,299.0,1.24,1,297.76,14.69
4,61,2025-01-13,MNQ,2027.0,SHORT,-90.5,1.24,1,-91.74,-4.53


In [10]:
performance.columns

Index(['id', 'date', 'ticker', 'margin', 'direction', 'gross_profit',
       'commissions', 'algo_version', 'net_profit', 'return'],
      dtype='object')

In [11]:
if len(performance) > 5:
    print(performance.sample(5))

      id        date ticker  margin direction  gross_profit  commissions  \
4     61  2025-01-13    MNQ  2027.0     SHORT         -90.5         1.24   
129  195  2025-06-05    MNQ  2027.0     SHORT         320.5         1.24   
144  210  2025-06-18    MNQ  2027.0      LONG        -139.5         1.24   
117  183  2025-05-23    MNQ  2027.0      LONG          70.5         1.24   
101  167  2025-05-07    MNQ  2027.0      LONG        -108.0         1.24   

     algo_version  net_profit  return  
4               1      -91.74   -4.53  
129             2      319.26   15.75  
144             2     -140.74   -6.94  
117             2       69.26    3.42  
101             2     -109.24   -5.39  


In [12]:
performance.describe()

,id,margin,gross_profit,commissions,algo_version,net_profit,return
count,151.000000,151.0,151.000000,1.510000e+02,151.000000,151.000000,151.000000
mean,139.251656,2027.0,-7.304636,1.240000e+00,1.847682,-8.544636,-0.421589
std,46.253103,0.0,216.181991,1.336701e-15,0.360525,216.181991,10.665116
min,57.000000,2027.0,-363.000000,1.240000e+00,1.000000,-364.240000,-17.970000
25%,103.500000,2027.0,-155.250000,1.240000e+00,2.000000,-156.490000,-7.720000
50%,141.000000,2027.0,-108.000000,1.240000e+00,2.000000,-109.240000,-5.390000
75%,178.500000,2027.0,190.250000,1.240000e+00,2.000000,189.010000,9.325000
max,216.000000,2027.0,667.500000,1.240000e+00,2.000000,666.260000,32.870000


Before getting into some of the better visualizations of the data, let's just look at some general imporant information in the data.

## Algorithm Data Summary

In [13]:
gross_profit_loss = performance['gross_profit'].sum()
pnl = 'profit'
if gross_profit_loss < 0:
    pnl = 'loss'
net_profit_loss = performance['net_profit'].sum()
commissions = performance['commissions'].sum()
win_df = performance[performance['net_profit'] > 0]
loss_df = performance[performance['net_profit'] < 0]
wins = len(win_df)
avg_winner = win_df['net_profit'].mean()
losses = len(loss_df)
avg_loser = loss_df['net_profit'].mean()
net_return = performance['return'].sum()
consecutive_profit_count = 0
max_cons_profit = 0
consecutive_loss_count = 0
max_cons_loss = 0
performance = performance.reset_index()
if len(performance) > 1:
    for i in range(len(performance)):
        if performance.loc[i,'net_profit'] > 0:
            consecutive_profit_count += 1
            if consecutive_profit_count > max_cons_profit:
                max_cons_profit = consecutive_profit_count
        else:
            consecutive_profit_count = 0
    for i in range(len(performance)):
        if performance.loc[i,'net_profit'] < 0:
            consecutive_loss_count += 1
            if consecutive_loss_count > max_cons_loss:
                max_cons_loss = consecutive_loss_count
        else:
            consecutive_loss_count = 0


print(f"You have taken {len(performance)} trades with the {algo} version {algo_version} Algorithm for a net return of {np.round(net_return,2)}% to date")
print(f"\nThis has resulted in a gross {pnl} of ${np.round(gross_profit_loss,2)} and you have paid ${np.round(commissions,2)} in commissions resulting in a net {pnl} of ${np.round(net_profit_loss,2)}")
print(f"\nYou have had a total of {wins} profitable trades and {losses} losing trades for a batting average of {np.round(wins/len(performance)*100,2)}%")
print(f"\nYour average winner is ${np.round(avg_winner,2)} and your average loser is ${np.round(avg_loser,2)} for an average reward/risk of {np.round(avg_winner/abs(avg_loser),1)}R")
print(f"\nYour best run so far is {max_cons_profit} winning trades in a row and your worst run is {max_cons_loss} losing trades in a row")

You have taken 151 trades with the  version None Algorithm for a net return of -63.66% to date

This has resulted in a gross loss of $-1103.0 and you have paid $187.24 in commissions resulting in a net loss of $-1290.24

You have had a total of 51 profitable trades and 100 losing trades for a batting average of 33.77%

Your average winner is $268.04 and your average loser is $-149.6 for an average reward/risk of 1.8R

Your best run so far is 4 winning trades in a row and your worst run is 7 losing trades in a row


## Data Visualizations

Let's take a look at the breakdown of trading days so far in a simple bar chart where green bars are profitable days adn red bars are drawdowns. This will give us a genearl idea of how well the algorithm has been performing. Large red streaks are indicative of big draw downs and should raise red flags.

In [24]:
#I'm getting future warnings from the altair package, so I'm going to supress them for now
#until altair releases an update
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

rrg.resampled_pnl_visualization(performance, algo, 10, 'D')

alt.LayerChart(...)

Let's look on a more macro scale of monthly performance of the algorithm.

In [28]:
#We can resample the data for week ('W'), month ('M'), quarter ('Q'), or year ('Y')
rrg.resampled_pnl_visualization(performance,algo, 30,'W')

alt.LayerChart(...)

Let's also create a profit loss curve to track our overall return (both $ and %). We will do this by adding a columns that takes the cumulative sum of both net_profit in US dollars and % return. 

In [16]:
#Let's look at our cumulative PnL Curve. We can do by Trade (resample_period = None: default)
#Or we can resample by day, week, month, year etc.
graph_1 = rrg.cummulative_pnlcurve_visualization(performance,algo,resample_period='D')
graph_1

alt.LayerChart(...)

# Version Comparison

Let's run all of the data based on algorithm Version2 and combine the graphs for visual purposes.

In [ ]:
if algo_version != 'None':
    performance_2 = performance_2[performance_2['algo_version'] == algo_version].reset_index()
else:
    pass

#Let's calcualte some additonal metrics quickly for analysis
performance_2['net_profit'] = performance_2['gross_profit'] - performance_2['commissions']
performance_2['return'] = np.round(performance_2['net_profit']/performance_2['margin'] *100,2)

performance_2.head()

,id,date,ticker,margin,direction,gross_profit,commissions,algo_version,net_profit,return
0,57,2025-01-06,MNQ,2027.0,LONG,-126.0,1.24,1,-127.24,-6.28
1,58,2025-01-07,MNQ,2027.0,SHORT,234.5,1.24,1,233.26,11.51
2,59,2025-01-08,MNQ,2027.0,LONG,-124.5,1.24,1,-125.74,-6.20
3,60,2025-01-10,MNQ,2027.0,SHORT,299.0,1.24,1,297.76,14.69
4,61,2025-01-13,MNQ,2027.0,SHORT,-90.5,1.24,1,-91.74,-4.53


In [19]:
performance.head()

,index,id,date,ticker,margin,direction,gross_profit,commissions,algo_version,net_profit,return
0,0,57,2025-01-06,MNQ,2027.0,LONG,-126.0,1.24,1,-127.24,-6.28
1,1,58,2025-01-07,MNQ,2027.0,SHORT,234.5,1.24,1,233.26,11.51
2,2,59,2025-01-08,MNQ,2027.0,LONG,-124.5,1.24,1,-125.74,-6.20
3,3,60,2025-01-10,MNQ,2027.0,SHORT,299.0,1.24,1,297.76,14.69
4,4,61,2025-01-13,MNQ,2027.0,SHORT,-90.5,1.24,1,-91.74,-4.53


In [20]:
gross_profit_loss = performance_2['gross_profit'].sum()
pnl = 'profit'
if gross_profit_loss < 0:
    pnl = 'loss'
net_profit_loss = performance_2['net_profit'].sum()
commissions = performance_2['commissions'].sum()
win_df = performance_2[performance_2['net_profit'] > 0]
loss_df = performance_2[performance_2['net_profit'] < 0]
wins = len(win_df)
avg_winner = win_df['net_profit'].mean()
losses = len(loss_df)
avg_loser = loss_df['net_profit'].mean()
net_return = performance_2['return'].sum()
consecutive_profit_count = 0
max_cons_profit = 0
consecutive_loss_count = 0
max_cons_loss = 0
performance_2 = performance_2.reset_index()
if len(performance_2) > 1:
    for i in range(len(performance_2)):
        if performance_2.loc[i,'net_profit'] > 0:
            consecutive_profit_count += 1
            if consecutive_profit_count > max_cons_profit:
                max_cons_profit = consecutive_profit_count
        else:
            consecutive_profit_count = 0
    for i in range(len(performance_2)):
        if performance_2.loc[i,'net_profit'] < 0:
            consecutive_loss_count += 1
            if consecutive_loss_count > max_cons_loss:
                max_cons_loss = consecutive_loss_count
        else:
            consecutive_loss_count = 0


print(f"You have taken {len(performance_2)} trades with the {algo} version {algo_version} Algorithm for a net return of {np.round(net_return,2)}% to date")
print(f"\nThis has resulted in a gross {pnl} of ${np.round(gross_profit_loss,2)} and you have paid ${np.round(commissions,2)} in commissions resulting in a net {pnl} of ${np.round(net_profit_loss,2)}")
print(f"\nYou have had a total of {wins} profitable trades and {losses} losing trades for a batting average of {np.round(wins/len(performance_2)*100,2)}%")
print(f"\nYour average winner is ${np.round(avg_winner,2)} and your average loser is ${np.round(avg_loser,2)} for an average reward/risk of {np.round(avg_winner/abs(avg_loser),1)}R")
print(f"\nYour best run so far is {max_cons_profit} winning trades in a row and your worst run is {max_cons_loss} losing trades in a row")

You have taken 112 trades with the  version None Algorithm for a net return of -123.73% to date

This has resulted in a gross loss of $-2369.0 and you have paid $138.88 in commissions resulting in a net loss of $-2507.88

You have had a total of 34 profitable trades and 78 losing trades for a batting average of 30.36%

Your average winner is $275.5 and your average loser is $-152.24 for an average reward/risk of 1.8R

Your best run so far is 3 winning trades in a row and your worst run is 9 losing trades in a row


In [21]:
rrg.cummulative_pnlcurve_visualization_comparison(performance, performance_2, algo, resample_period='D')

alt.LayerChart(...)